# 5 pandas入门

pandas 是本书后续内容的首选库。它含有使数据清洗和分析工作变得更快更简 单的数据结构和操作工具。
pandas 经常和其它工具一同使用，如数值计算工具 NumPy 和 SciPy，分析库 statsmodels 和 scikit-learn，和数据可视化库 matplotlib。pandas 是基于 NumPy 数组构建的，特别是基于数组的函数和不使 用 for 循环的数据处理。
虽然 pandas 采用了大量的 NumPy 编码风格，但二者最大的不同是 pandas 是专 门为处理表格和混杂数据设计的。而 NumPy 更适合处理统一的数值数组数据。

In [1]:
import pandas as pd
from pandas import Series, DataFrame

## 5.1 pandas的数据结构介绍

要使用 pandas，你首先就得熟悉它的两个主要数据结构：Series 和 DataFrame。虽然它们并不能解决所有问题，但它们为大多数应用提供了一种可 靠的、易于使用的基础

### Series

Series 是一种类似于一维数组的对象，它由一组数据（各种 NumPy 数据类型） 以及一组与之相关的数据标签（即索引）组成。仅由一组数据即可产生最简单 的 Series：

In [2]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

Series 的字符串表现形式为：索引在左边，值在右边。由于我们没有为数据指 定索引，于是会自动创建一个 0 到 N-1（N 为数据的长度）的整数型索引。你可 以通过 Series 的 values 和 index 属性获取其数组表示形式和索引对象

In [3]:
obj.values

array([ 4,  7, -5,  3])

In [4]:
obj.index

RangeIndex(start=0, stop=4, step=1)

通常，我们希望所创建的 Series 带有一个可以对各个数据点进行标记的索引

In [6]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

与普通 NumPy 数组相比，你可以通过索引的方式选取 Series 中的单个或一组 值：

In [7]:
obj2['a']

-5

In [8]:
obj2['d'] = 6
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [9]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

使用 NumPy 函数或类似 NumPy 的运算（如根据布尔型数组进行过滤、标量乘 法、应用数学函数等）都会保留索引值的链接：

In [10]:
obj2[obj2  > 0]

d    6
b    7
c    3
dtype: int64

In [11]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [12]:
import numpy as np
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

In [13]:
# 还可以将 Series 看成是一个定长的有序字典，因为它是索引值到数据值的一个 映射。它可以用在许多原本需要字典参数的函数中：
'b' in obj2

True

In [14]:
# 如果数据被存放在一个 Python 字典中，也可以直接通过这个字典来创建 Series：
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [15]:
obj3.index

Index(['Ohio', 'Texas', 'Oregon', 'Utah'], dtype='object')

In [17]:
# 如果只传入一个字典，则结果 Series 中的索引就是原字典的键（有序排列）。 你可以传入排好序的字典的键以改变顺序：
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata,index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [18]:
# 在这个例子中，sdata 中跟 states 索引相匹配的那 3 个值会被找出来并放到相 应的位置上，但由于"California"所对应的 sdata 值找不到，所以其结果就为 NaN（即“非数字”（not a number），在 pandas 中，它用于表示缺失或 NA 值）。因为‘Utah’不在 states 中，它被从结果中除去

使用缺失（missing）或 NA 表示缺失数据。pandas 的 isnull 和 notnull 函数可用于检测缺失数据

In [19]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [20]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [21]:
# Series 也有类似的实例方法
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

对于许多应用而言，Series 最重要的一个功能是，它会根据运算的索引标签自 动对齐数据

In [22]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [23]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [24]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Series 对象本身及其索引都有一个 name 属性，该属性跟 pandas 其他的关键功 能关系非常密切：

In [25]:
obj4.name = 'population'

In [26]:
obj4.index.name = 'state'

In [27]:
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

Series 的索引可以通过赋值的方式就地修改

In [28]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [29]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']

In [30]:
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### DataFrame

DataFrame 是一个表格型的数据结构，它含有一组有序的列，每列可以是不同 的值类型（数值、字符串、布尔值等）。DataFrame 既有行索引也有列索引， 它可以被看做由 Series 组成的字典（共用同一个索引）。DataFrame 中的数据 是以一个或多个二维块存放的（而不是列表、字典或别的一维数据结构）。有 关 DataFrame 内部的技术细节远远超出了本书所讨论的范围。

笔记：虽然 DataFrame 是以二维结构保存数据的，但你仍然可以轻松地将其表 示为更高维度的数据（层次化索引的表格型结构，这是 pandas 中许多高级数据 处理功能的关键要素，我们会在第 8 章讨论这个问题）。

建 DataFrame 的办法有很多，最常用的一种是直接传入一个由等长列表或 NumPy 数组组成的字典：

In [33]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
'year': [2000, 2001, 2002, 2001, 2002, 2003],
'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)
frame
#结果 DataFrame 会自动加上索引（跟 Series 一样），且全部列会被有序排列

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [34]:
# 对于特别大的 DataFrame，head 方法会选取前五行
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


In [35]:
# 如果指定了列序列，则 DataFrame 的列就会按照指定顺序进行排列：
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


In [37]:
# 如果传入的列在数据中找不到，就会在结果中产生缺失值：
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
index=['one', 'two', 'three', 'four', 'five', 'six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [38]:
frame2.columns 
# 通过类似字典标记的方式或属性的方式，可以将 DataFrame 的列获取为一个 Series：

Index(['year', 'state', 'pop', 'debt'], dtype='object')

In [39]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

笔记：IPython 提供了类似属性的访问（即 frame2.year）和 tab 补全。 frame2[column]适用于任何列的名，但是 frame2.column 只有在列名是一个合 理的 Python 变量名时才适用。

注意，返回的 Series 拥有原 DataFrame 相同的索引，且其 name 属性也已经被 相应地设置好了。

行也可以通过位置或名称的方式进行获取，比如用 loc 属性（稍后将对此进行 详细讲解）：

In [40]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

列可以通过赋值的方式进行修改。例如，我们可以给那个空的"debt"列赋上一 个标量值或一组值：

In [41]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


In [43]:
frame2['debt'] = np.arange(6.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


将列表或数组赋值给某个列时，其长度必须跟 DataFrame 的长度相匹配。如果 赋值的是一个 Series，就会精确匹配 DataFrame 的索引，所有的空位都将被填 上缺失值：

In [45]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
val

two    -1.2
four   -1.5
five   -1.7
dtype: float64

In [47]:
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


为不存在的列赋值会创建出一个新列。关键字 del 用于删除列。

In [49]:
# 作为 del 的例子，我先添加一个新的布尔值的列，state 是否为'Ohio'：
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


注意：不能用 frame2.eastern 创建新的列。

In [50]:
# del 方法可以用来删除这列：
del frame2['eastern']

In [52]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

注意：通过索引方式返回的列只是相应数据的视图而已，并不是副本。因此， 对返回的 Series 所做的任何就地修改全都会反映到源 DataFrame 上。通过 Series 的 copy 方法即可指定复制列。

In [53]:
#另一种常见的数据形式是嵌套字典
pop = {'Nevada': {2001: 2.4, 2002: 2.9}, 'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

In [54]:
# 如果嵌套字典传给 DataFrame，pandas 就会被解释为：外层字典的键作为列， 内层键则作为行索引：
frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


你也可以使用类似 NumPy 数组的方法，对 DataFrame 进行转置（交换行和 列）：

In [55]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


In [56]:
# 内层字典的键会被合并、排序以形成最终的索引。如果明确指定了索引，则不 会这样：
pd.DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


In [58]:
# 由 Series 组成的字典差不多也是一样的用法：
pdata = {'Ohio': frame3['Ohio'][:-1], 'Nevada': frame3['Nevada'][:2]}
pd.DataFrame(pdata)

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4


In [59]:
# 如果设置了 DataFrame 的 index 和 columns 的 name 属性，则这些信息也会被显 示出来：
frame3.index.name = 'year'
frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


跟 Series 一样，values 属性也会以二维 ndarray 的形式返回 DataFrame 中的 数据：

In [60]:
frame3.values

array([[nan, 1.5],
       [2.4, 1.7],
       [2.9, 3.6]])

In [62]:
frame3.values.shape

(3, 2)

如果 DataFrame 各列的数据类型不同，则值数组的 dtype 就会选用能兼容所有 列的数据类型：

In [61]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

In [63]:
frame2.values.shape

(6, 4)

### 索引对象

pandas 的索引对象负责管理轴标签和其他元数据（比如轴名称等）。构建 Series 或 DataFrame 时，所用到的任何数组或其他序列的标签都会被转换成一 个 Index：

In [65]:
obj = pd.Series(range(3),  index=['a', 'b', 'c'])
obj

a    0
b    1
c    2
dtype: int64

In [67]:
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [68]:
index[1:]

Index(['b', 'c'], dtype='object')

Index 对象是不可变的，因此用户不能对其进行修改：

In [69]:
index[1] = 'd' # TypeError: Index does not support mutable operations

TypeError: Index does not support mutable operations

不可变可以使 Index 对象在多个数据结构之间安全共享

In [70]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [73]:
obj2 = pd.Series([1.5,-2.6,0], index=labels)
obj2

0    1.5
1   -2.6
2    0.0
dtype: float64

In [74]:
obj2.index is labels

True

注意：虽然用户不需要经常使用 Index 的功能，但是因为一些操作会生成包含 被索引化的数据，理解它们的工作原理是很重要的。

In [75]:
# 除了类似于数组，Index 的功能也类似一个固定大小的集合：
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [77]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object', name='state')

In [78]:
'Ohio' in frame3.columns

True

In [79]:
2003 in frame3.columns

False

In [83]:
# 与 python 的集合不同，pandas 的 Index 可以包含重复的标签
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

选择重复的标签，会显示所有的结果。

每个索引都有一些方法和属性，它们可用于设置逻辑并回答有关该索引所包含 的数据的常见问题

## 5.2 基本功能

本节中，我将介绍操作 Series 和 DataFrame 中的数据的基本手段。后续章节将 更加深入地挖掘 pandas 在数据分析和处理方面的功能。本书不是 pandas 库的 详尽文档，主要关注的是最重要的功能，那些不大常用的内容（也就是那些更 深奥的内容）就交给你自己去摸索吧

### 重新索引

pandas 对象的一个重要方法是 reindex，其作用是创建一个新对象，它的数据 符合新的索引。看下面的例子

In [84]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [85]:
#用该 Series 的 reindex 将会根据新索引进行重排。如果某个索引值当前不存 在，就引入缺失值：
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [87]:
# 对于时间序列这样的有序数据，重新索引时可能需要做一些插值处理。method 选项即可达到此目的，例如，使用 ffill 可以实现前向值填充
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [88]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

借助 DataFrame，reindex 可以修改（行）索引和列。只传递一个序列时，会重 新索引结果的行：

In [89]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)), 
                     index=['a', 'c', 'd'], 
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [91]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [92]:
# 列可以用 columns 关键字重新索引：
states = ['Texas', 'Utah', 'California']

In [93]:
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


### 丢弃指定轴上的项

丢弃某条轴上的一个或多个项很简单，只要有一个索引数组或列表即可。由于 需要执行一些数据整理和集合逻辑，所以 drop 方法返回的是一个在指定轴上删 除了指定值的新对象

In [104]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [95]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [96]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

对于 DataFrame，可以删除任意轴上的索引值。为了演示，先新建一个 DataFrame 例子：

In [97]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)), 
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [98]:
# 用标签序列调用 drop 会从行标签（axis 0）删除值：
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [100]:
# 通过传递 axis=1 或 axis='columns'可以删除列的值：
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [101]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


许多函数，如 drop，会修改 Series 或 DataFrame 的大小或形状，可以就地修 改对象，不会返回新的对象：

In [106]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

小心使用 inplace，它会销毁所有被删除的数据